In [ ]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port=5433,
    user="ubuntu",          # ← replace with your actual DB user
    dbname="brti"             # ← replace with your actual DB name
)
cur = conn.cursor()
cur.execute("SELECT NOW();")
print(cur.fetchone())
print(cur.fetchone()[0])


(datetime.datetime(2025, 6, 14, 16, 18, 45, 856974, tzinfo=datetime.timezone.utc),)


In [9]:
import psycopg2
from datetime import timezone
from zoneinfo import ZoneInfo  # Python 3.9+
import datetime

# === CONFIGURATION ===
DB_HOST = "localhost"
DB_PORT = 5433  # this matches your SSH tunnel
DB_NAME = "brti"  # change if yours is different
DB_USER = "ubuntu"  # change to your DB username

# === CONNECT TO DB ===
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME,
        user=DB_USER,
    )
    cur = conn.cursor()

    # === FETCH LATEST PRICE ===
    cur.execute("SELECT * FROM brti_prices ORDER BY timestamp DESC LIMIT 1;")
    row = cur.fetchone()

    if row:
        price, ts_utc = row[0], row[1]
        ts_est = ts_utc.astimezone(ZoneInfo("US/Eastern"))

        print("📈 Latest BRTI Price")
        print("-------------------------")
        print(f"💲 Price:        ${price:,.2f}")
        print(f"🕒 UTC Time:     {ts_utc.strftime('%Y-%m-%d %H:%M:%S.%f %Z')}")
        print(f"🕒 Eastern Time: {ts_est.strftime('%Y-%m-%d %H:%M:%S.%f %Z')}")
    else:
        print("⚠️ No data found in the brti_prices table.")

    cur.close()
    conn.close()

except Exception as e:
    print("❌ Error:", e)


❌ Error: 'decimal.Decimal' object has no attribute 'astimezone'
